In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
from matplotlib import pyplot as plt

import sqlite3

from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, StandardScaler

import tensorflow as tf

In [2]:
tf.__version__

'2.3.1'

In [3]:
data = pd.read_csv("data/parenting_dataset_asli.csv", index_col="NO")
data.drop(columns=["Responden"], inplace=True)
data.head()

,AE1,AE2,AE3,AE4,AE5,AE6,AE7,AE8,AE9,AE10,...,PE2,PE3,PE4,PE5,PE6,PE7,PE8,PE9,PE10,Parenting_style
NO,,,,,,,,,,,,,,,,,,,,,
1,1,5,2,5,1,4,1,4,1,5,...,4,3,1,4,1,2,4,1,1,Authoration
2,1,4,2,4,2,5,1,4,5,5,...,2,3,2,3,4,3,2,2,1,Authorative
3,2,4,4,5,5,4,2,5,5,5,...,4,3,4,4,4,4,4,3,3,Authorative
4,1,3,2,4,2,4,1,3,2,5,...,5,2,4,3,3,3,5,5,4,Permissive
5,1,1,3,3,1,1,1,5,3,3,...,5,1,4,3,4,3,4,4,3,Permissive


In [8]:
# fix random seed for reproducibility
seed = 42
np.random.seed(seed)

In [4]:
X = data.drop(['Parenting_style'], axis = 1)
Y = data['Parenting_style']

print("Shape of Input  features: {}".format(X.shape))
print("Shape of Output features: {}".format(Y.shape))

Shape of Input  features: (500, 52)
Shape of Output features: (500,)


In [5]:
Y.value_counts()

Authorative    182
Authoration    163
Permissive     155
Name: Parenting_style, dtype: int64

In [12]:
X.head()

,AE1,AE2,AE3,AE4,AE5,AE6,AE7,AE8,AE9,AE10,...,PE1,PE2,PE3,PE4,PE5,PE6,PE7,PE8,PE9,PE10
NO,,,,,,,,,,,,,,,,,,,,,
1,1,5,2,5,1,4,1,4,1,5,...,1,4,3,1,4,1,2,4,1,1
2,1,4,2,4,2,5,1,4,5,5,...,1,2,3,2,3,4,3,2,2,1
3,2,4,4,5,5,4,2,5,5,5,...,4,4,3,4,4,4,4,4,3,3
4,1,3,2,4,2,4,1,3,2,5,...,5,5,2,4,3,3,3,5,5,4
5,1,1,3,3,1,1,1,5,3,3,...,2,5,1,4,3,4,3,4,4,3


In [6]:
lbl_clf = LabelEncoder()
Y_encoded = lbl_clf.fit_transform(Y)

#Keras requires your output feature to be one-hot encoded values.
Y_final = tf.keras.utils.to_categorical(Y_encoded)

print("Therefore, our final shape of output feature will be {}".format(Y_final.shape))

Therefore, our final shape of output feature will be (500, 3)


In [9]:
x_train, x_test, y_train, y_test = train_test_split(X, Y_final, test_size=0.2, random_state=seed, stratify=Y_encoded, shuffle=True)

print("Training Input shape\t: {}".format(x_train.shape))
print("Testing Input shape\t: {}".format(x_test.shape))
print("Training Output shape\t: {}".format(y_train.shape))
print("Testing Output shape\t: {}".format(y_test.shape))

Training Input shape	: (400, 52)
Testing Input shape	: (100, 52)
Training Output shape	: (400, 3)
Testing Output shape	: (100, 3)


In [13]:
std_clf = StandardScaler()
x_train_new = std_clf.fit_transform(x_train)
x_test_new = std_clf.transform(x_test)

In [41]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Dense(16, input_dim=52, activation='relu', kernel_initializer='he_normal', 
                                kernel_regularizer=tf.keras.regularizers.l2(0.01)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(8, activation='relu', kernel_initializer='he_normal', 
                                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.001, l2=0.001)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.2))
model.add(tf.keras.layers.Dense(8, activation='relu', kernel_initializer='he_normal', 
                                kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.001, l2=0.001)))
model.add(tf.keras.layers.Dense(3, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

model_history = model.fit(x_train_new, y_train, epochs=200, batch_size=10)

Epoch 1/200
40/40 [==============================] - 0s 1ms/step - loss: 1.7011 - accuracy: 0.3925
Epoch 2/200
40/40 [==============================] - 0s 975us/step - loss: 1.5625 - accuracy: 0.3950
Epoch 3/200
40/40 [==============================] - 0s 1ms/step - loss: 1.4641 - accuracy: 0.4350
Epoch 4/200
40/40 [==============================] - 0s 975us/step - loss: 1.4263 - accuracy: 0.4775
Epoch 5/200
40/40 [==============================] - 0s 1ms/step - loss: 1.3924 - accuracy: 0.4875
Epoch 6/200
40/40 [==============================] - 0s 1ms/step - loss: 1.3610 - accuracy: 0.4825
Epoch 7/200
40/40 [==============================] - 0s 1ms/step - loss: 1.3453 - accuracy: 0.5000
Epoch 8/200
40/40 [==============================] - 0s 1ms/step - loss: 1.3189 - accuracy: 0.5425
Epoch 9/200
40/40 [==============================] - 0s 1ms/step - loss: 1.3298 - accuracy: 0.5350
Epoch 10/200
40/40 [==============================] - 0s 1ms/step - loss: 1.2915 - accuracy: 0.5425
Epoch

40/40 [==============================] - 0s 1ms/step - loss: 0.4770 - accuracy: 0.8800
Epoch 83/200
40/40 [==============================] - 0s 1ms/step - loss: 0.4947 - accuracy: 0.8600
Epoch 84/200
40/40 [==============================] - 0s 1ms/step - loss: 0.4596 - accuracy: 0.8875
Epoch 85/200
40/40 [==============================] - 0s 1ms/step - loss: 0.4868 - accuracy: 0.8950
Epoch 86/200
40/40 [==============================] - 0s 975us/step - loss: 0.4261 - accuracy: 0.9000
Epoch 87/200
40/40 [==============================] - 0s 1ms/step - loss: 0.4692 - accuracy: 0.8750
Epoch 88/200
40/40 [==============================] - 0s 1ms/step - loss: 0.4383 - accuracy: 0.8950
Epoch 89/200
40/40 [==============================] - 0s 1ms/step - loss: 0.5070 - accuracy: 0.8725
Epoch 90/200
40/40 [==============================] - 0s 975us/step - loss: 0.3817 - accuracy: 0.9250
Epoch 91/200
40/40 [==============================] - 0s 1ms/step - loss: 0.4936 - accuracy: 0.8800
Epoch 92/

40/40 [==============================] - 0s 1ms/step - loss: 0.3463 - accuracy: 0.9275
Epoch 164/200
40/40 [==============================] - 0s 975us/step - loss: 0.3199 - accuracy: 0.9275
Epoch 165/200
40/40 [==============================] - 0s 1ms/step - loss: 0.2721 - accuracy: 0.9575
Epoch 166/200
40/40 [==============================] - 0s 1ms/step - loss: 0.3140 - accuracy: 0.9300
Epoch 167/200
40/40 [==============================] - 0s 1ms/step - loss: 0.3298 - accuracy: 0.9100
Epoch 168/200
40/40 [==============================] - 0s 1ms/step - loss: 0.2791 - accuracy: 0.9475
Epoch 169/200
40/40 [==============================] - 0s 950us/step - loss: 0.3792 - accuracy: 0.8875
Epoch 170/200
40/40 [==============================] - 0s 975us/step - loss: 0.3212 - accuracy: 0.9350
Epoch 171/200
40/40 [==============================] - 0s 975us/step - loss: 0.3164 - accuracy: 0.9250
Epoch 172/200
40/40 [==============================] - 0s 1ms/step - loss: 0.3092 - accuracy: 0.9

In [42]:
model.evaluate(x_test_new, y_test)

4/4 [==============================] - 0s 1ms/step - loss: 0.6066 - accuracy: 0.8200


[0.60655677318573, 0.8199999928474426]

In [ ]:
# EXERCISE: Use the tf.saved_model API to save your model in the SavedModel format. 
export_dir = 'saved_model/1'

tf.saved_model.save(model, export_dir)

In [ ]:
# Select mode of optimization
mode = "Speed" 

if mode == 'Storage':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_SIZE
elif mode == 'Speed':
    optimization = tf.lite.Optimize.OPTIMIZE_FOR_LATENCY
else:
    optimization = tf.lite.Optimize.DEFAULT